In [2]:
en_dict_path = r'\\wsl$\Ubuntu-20.04\usr\share\dict\british-english'
per_dict_path= r'C:\Dev\data\persian_words.txt'
#per_dict_path= r'C:\Dev\data\persian_word_larger.txt'
lang_dict = {'en': en_dict_path, 'per': per_dict_path}
encoding = {'en': 'utf-8', 'per': 'utf-8'}
normalizer_dict = {}
word_len = 5


def read_file(lang='en'):
    global lang_dict
    with open(lang_dict[lang], 'r', encoding=encoding[lang]) as f:
        words = f.readlines()
        return words
    return None


def normalizer(word_list: list, acceptable_letter=[], not_acceptable_letter=[]) -> list:
    word_list = [word.strip().lower() for word in word_list]
    word_list = [word for word in word_list  if len(word) == word_len]
    normal_words = [word for word in word_list if all(letter in acceptable_letter for letter in word)]
    normal_words = [word for word in normal_words if not any(letter in word for letter in not_acceptable_letter)]
    return normal_words


def english_normalizer(word_list: list) -> list:
    legal_letters = [chr(c) for c in range(ord('a'), ord('z')+1)]
    return normalizer(word_list, legal_letters)


def persian_normalizer(word_list: list) -> list:
    legal_letters = list('ابپتثجچحخدذرزژسشصضعغفقکگلمنوهیطظآئآأؤءكژَٔ')
    return normalizer(word_list, legal_letters)


def letter_freqency_calc(word_list: list) -> dict:
    freq_dict = {}
    for word in word_list:
        for letter in word:
            if letter in freq_dict:
                freq_dict[letter] += 1
            else:
                freq_dict[letter] = 1
    return freq_dict


def point_calc_(word_list:list, letter_grade: dict) -> dict:
    n = len(word_list)
    word_point = {word: (1.0*sum(letter_grade[l] for l in word))/n for word in word_list}
    return word_point


def sort_list_and_fork(word_dict: dict) -> (list, list):
    word_dict = {word: point for word, point in sorted(word_dict.items(), key=lambda item:item[1], reverse=True)}
    word_list = [word for word, point in word_dict.items()]
    unique_list = [word for word, point in word_dict.items() if len(set(list(word))) == len(word)]
    return word_list, unique_list


def get_next_suggest(word_list:list, has=[], has_indx={}, not_have=[], skip=0):
    latest = ''
    for word in word_list:
        is_letter_cond = all(letter in word for letter in has) and (not any(letter in word for letter in not_have))
        position_cond = all(word[i] == letter for i, letter in has_indx.items())
        if  is_letter_cond and position_cond:
            latest = word
            if skip == 0:
                return word
            else:
                skip -= 1
    return latest


def process_word(word_list: list) -> (list, list):
    letter_dict = letter_freqency_calc(word_list)
    word_points = point_calc_(word_list, letter_dict)
    return sort_list_and_fork(word_points)
normalizer_dict = {'en': lambda w: english_normalizer(w), 'per': lambda w: persian_normalizer(w)}

In [3]:
lang = 'en'

word_list = read_file(lang)
word_list = normalizer_dict[lang](word_list)
wlist, ulist = process_word(word_list)

In [7]:
has = list()
has_indx = {}
not_have = list()
print("unique=\t",get_next_suggest(ulist, has, has_indx, not_have))
print("Guess=\t",get_next_suggest(wlist, has, has_indx, not_have))

unique=	 
Guess=	 loony
